In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()
bucket = sess.default_bucket()
region_name = boto3.Session().region_name
prefix = 'gcr_sagemaker_workshop/classification_regression/linear_learner'

In [ ]:
# import numpy and pandas libraries for working with data
import numpy as np
import pandas as pd# Read in csv and store in a pandas dataframe
df = pd.read_csv('./data/linear-learner-MatchData.csv', sep=',' )

To verify that the csv was read correctly you can execute df.head() to get a list of the top 5 entries in your dataframe.

In [ ]:
df.head(5)

The csv has a lot of data that we don’t need right now, we should create a dataframe with only the information we care about. Let’s create a new pandas df with only the columns we require for the excercise.

In [ ]:
# keep player name for readability and manual checking
data = df.loc[:, ['player_name', 'K', 'H', 'M', 'T', 'G', 'B', 'HO', 'FF', 'FA', 'AF']]# Remove player name as it is irrelevant for calcs
playerStats = data.loc[:, ['K', 'H','M','T','G','B','HO','FF','FA']]# confirm we got the data we wanted
data.head(10)

We now have an array of all the relevant player stats for every game of AFL in the 2018 season so far as well as the Fantasy Points that the player scored.

Now AFL fantasy points are calculated by the following formula:

Kick (3), Handball (2), Mark (3), Tackle (4), Goal (6), Behind (1), Hit Out (1), Free Kick For (1), Free Kick Against (-3)

I’ve ordered these in the same order as our array so that we can create a weightings array in this order.

In [ ]:
weightings = [3, 2, 3, 4, 6, 1, 1, 1, -3]

Before we run any ML algorithms we should verify that our data and weighting array are valid. Lets write a simple function to confirm this.

This function will take an array of player stats and a vector of weights and multiply each stat by the relevant weight and sum them together to give us calculated Fantasy Points.

In [ ]:
def calculate_fantasy_points(playerStats, Weightings):
    return np.dot(playerStats, np.transpose(weightings))

Now we can calculate fantasy points based on the weightings vector we have created and verify that they are indeed the correct weights.

In [ ]:
# Calculate Fantasy Points
data['calculated'] = calculate_fantasy_points(playerStats, weightings)# Get the difference between actual points and predicted
data['diff'] = data['AF'] - calculate_fantasy_points(playerStats, weightings)# Take the sum of the difference over all data points and verify that is is zero
data['diff'].sum()

At this stage we see that indeed, the weighting vector we created above is correct and does generate the Fantasy Points we would expect. The next step is to see if the SageMaker Linear Learner can find that weighting vector if it was unknown to us.

##### Using SageMaker Linear Learner
The first thing we need to do is to prepare the data in a format that SageMaker can use. The Linear Learner requires a numpy array of type float32.

In [ ]:
# Kicks, handballs, goals etc
modelData = np.array(data.iloc[:, 1:10]).astype('float32')

# Actual Fantasy Points
target = np.array(data.iloc[:, 10]).astype('float32')

#Verify that the conversion worked
print(modelData[0])

Next we need to import some librarys to communicate with the ML instances
!!! Don't forget to change the bucket and prefix name as you like.!!!

In [ ]:
import io
import os
import sagemaker.amazon.common as smac

# Use the IO buffer as dataset is small
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, modelData, target)
buf.seek(0)

key = 'linearlearner'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)

s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))

output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(region_name, 'linear-learner')
print(container)

Now that we’ve done some setup and configuration, we can look at running the model.

In [ ]:
linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.c4.xlarge',
                                       output_path=output_location,
                                       sagemaker_session=sess)

We need to set some model parameters for this model. Specifically we need to tell the linear learner that we have 9 parameters to fit, that we want a regression model, and most importantly we do not want to normalise the data.

In [ ]:
linear.set_hyperparameters(feature_dim=9,
                           predictor_type='regressor',
                           normalize_data=False)

Now we are ready to deploy our model to an instance to run the linear learner and get results. To deploy this model we simply run the following codes.
This will take a couple of minutes to provision and run and will let you know when it’s done.

In [ ]:
linear.fit({'train': s3_train_data})

In [ ]:
linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

##### Accessing the results
Once the model has been trained, we can send new data to the model and obtain predictions. In this case we are just going to send it the training data back and see how close it got to finding the correct weights.

In [ ]:
# Set up
from sagemaker.predictor import csv_serializer, json_deserializer

linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

Pass all the data to the predictor and get all the results back

In [ ]:
predictions = []
for array in modelData:
    result = linear_predictor.predict(array)
    predictions += [r['score'] for r in result['predictions']]

In [ ]:
predictions = np.array(predictions)
# Push into our pandas dataframe
data['Predicted'] = predictions.astype(int)
data.head(5)

In [ ]:
# envaluation
y_predicted = data['Predicted']
y = data['calculated']

In [ ]:
def evaluationModel(y,y_pre):
    accuracy_cnt = 0
    for i,j in zip(y,y_pre):
        if abs(i-j) <= 1:
            accuracy_cnt += 1
    accuracy_rate = float(accuracy_cnt) / len(y_predicted)
    print ('test samples：{0}'.format(len(y_predicted)))
    print ('accurate samples：{0}'.format(accuracy_cnt))
    print ('accuracy rate：{0:.3f}'.format((accuracy_rate)))   

In [ ]:
evaluationModel(y, y_predicted)

The results were very close.
And Sagemaker make it possible to deliver the ease of setting up the model, and just a little domain knowledge required to run this simple regression.

##### (Optional) Delete the Endpoint

If you're ready to be done with this notebook, please run the delete_endpoint line in the cell below. This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.


In [ ]:
sess.delete_endpoint(linear_predictor.endpoint)